In [13]:
from vis_utils import VolumeVisualizer
from identification import get_points_on_sphere

In [2]:
import numpy as np
import networkx as nx
import pickle

In [3]:
visualize_steps = True 

In [4]:
def visualize_mask_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8), binary=True).visualize()
    
def visualize_mask_non_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8) * 255, binary=False).visualize()

In [5]:
def reconstruct_model(model_shape, graph: nx.Graph):
    mask = np.zeros(model_shape, dtype=np.uint8)

    for edge in graph.edges:
        node1, node2 = edge
        lw = graph.get_edge_data(node1, node2)['average_vessel_diameter'] 
        x1, y1, z1 = node1
        x2, y2, z2 = node2
        line_x = np.linspace(x1, x2, num=300, endpoint=True, dtype=np.int32)
        line_y = np.linspace(y1, y2, num=300, endpoint=True, dtype=np.int32)
        line_z = np.linspace(z1, z2, num=300, endpoint=True, dtype=np.int32)
        for i in range(len(line_x)):
            sphere_points = get_points_on_sphere((line_x[i], line_y[i], line_z[i]), lw)
            for point in sphere_points:
                px, py, pz = int(point[0]), int(point[1]), int(point[2])
                mask[px, py, pz] = 1

    for node in graph.nodes:
        x, y, z = node
        mask[x, y, z] = 1
    return mask

In [6]:
with open('clean_data_graph_full.pickle', 'rb') as file:
    G = pickle.load(file)

In [7]:
G

In [9]:
original_shape = (700, 500, 800) # zaladowac oryginalny obraz wyciagnac shape albo tylko shape wpisac
mask = reconstruct_model(original_shape, G)

In [9]:
np.save("graf_ciekawe_co_wyszlo.npy", mask)

In [10]:
# mask = np.load("graf_ciekawe_co_wyszlo.npy")

In [11]:
visualize_mask_bin(mask)

In [12]:
visualize_mask_non_bin(mask)